In [1]:

import pandas as pd
from numpy import cumsum, polyfit, log, sqrt, std, subtract
import numpy as np
import datetime
import statsmodels.tsa.stattools as ts
import os 
from pandas_datareader import data as pdr

In [2]:
#Arrays donde vamos a guardar los tickers que cumplan mean reversion al 99% o al 95%
mr_arr99 = []
mr_arr95 = [] 
mr_arr90 = []

#Pares de forex
table=pd.read_html('https://finance.yahoo.com/currencies/')
#Los tickers del S&P500
#table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

df = table[0] 
ticker_list=df['Symbol'].tolist() 
print(ticker_list)
#Elegimos fechas sobre los que haremos los estudios de las series 
start = datetime.datetime.now() - datetime.timedelta(days=2*365)
end = datetime.datetime.now() - datetime.timedelta(days=365)

['BTCUSD=X', 'ETHUSD=X', 'EURUSD=X', 'JPY=X', 'GBPUSD=X', 'AUDUSD=X', 'NZDUSD=X', 'EURJPY=X', 'GBPJPY=X', 'EURGBP=X', 'EURCAD=X', 'EURSEK=X', 'EURCHF=X', 'EURHUF=X', 'EURJPY=X', 'CNY=X', 'HKD=X', 'SGD=X', 'INR=X', 'MXN=X', 'PHP=X', 'IDR=X', 'THB=X', 'MYR=X', 'ZAR=X', 'RUB=X']


In [3]:
#Definir el exponente de HURST 

def hurst_exponent(data):
    """
    calculate HE on our time series data
    args:
        data: pandas dataframe of data
        lags: array of our lag range, default 2 to 20
    returns:
        float, HE value
    """
    lags = range(2,100)
    tau = [sqrt(std(subtract(data[lag:], data[:-lag]))) for lag in lags]
    m = np.polyfit(log(lags), log(tau), 1)
    hurst = m[0]*2
    return hurst


In [4]:
#Comprobamos ADF por cada ticker, y vamos guardando en los array 
for ticker in ticker_list: 
    try:        
        df= pdr.get_data_yahoo(ticker,start= start, end= end) 
        print(f'Analizando {ticker}...')
        df=df['Close']
        
        result= ts.adfuller(df,1)
        
        hurst = hurst_exponent(df.values)  
        
        # Añadimos los tickers a los arrays si pasan el test de Augmented Dickey Fuller 
        if  result[0] < result[4]['1%'] and hurst < 0.45 : 
            mr_arr99.append(f'{ticker}:  {hurst}')
            print(ticker)
        elif result[0] < result[4]['5%'] and hurst < 0.48 : 
            mr_arr95.append(f'{ticker}:  {hurst}')
            print(ticker)
        elif result[0] < result[4]['10%'] and hurst < 0.5 :  
            mr_arr90.append(f'{ticker}:  {hurst}') 
            print(ticker)
        else: 
            print('.')
    except:
        pass
    






Analizando BTCUSD=X...
.
Analizando ETHUSD=X...
.
Analizando EURUSD=X...
.
Analizando JPY=X...
.
Analizando GBPUSD=X...
.
Analizando AUDUSD=X...
.
Analizando NZDUSD=X...
.
Analizando EURJPY=X...
.
Analizando GBPJPY=X...
.
Analizando EURGBP=X...
.
Analizando EURCAD=X...
.
Analizando EURSEK=X...
.
Analizando EURCHF=X...
.
Analizando EURHUF=X...
.
Analizando EURJPY=X...
.
Analizando CNY=X...
.
Analizando HKD=X...
.
Analizando SGD=X...
.
Analizando INR=X...
INR=X
Analizando MXN=X...
.
Analizando PHP=X...
.
Analizando IDR=X...
IDR=X
Analizando THB=X...
.
Analizando MYR=X...
.
Analizando ZAR=X...
.
Analizando RUB=X...
.


In [5]:
print(mr_arr90)

[]


In [6]:
print(mr_arr95)

['INR=X:  0.2948573834716733']


In [7]:
print(mr_arr99)

['IDR=X:  0.2395958595837631']
